# Use the results from magnet training to compute TPR, FPR and AUC

In [217]:
import numpy as np
from scipy.stats import entropy
from scipy import ndimage

In [218]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [219]:
def train_fs(distances, train_fpr):
    selected_distance_idx = int(np.ceil(len(distances) * (1-train_fpr)))
    threshold = sorted(distances)[selected_distance_idx-1]
    threshold = threshold
    #print ("Threshold value: %f" % threshold)
    return threshold

In [220]:
def test(model, dataset, X, threshold):
    distances = get_distance_test(model, dataset, X)
    Y_pred = distances > threshold
    return Y_pred, distances

In [221]:
def compute_distance(model, dataset, X):
    distances = get_distance_test(model, dataset, X)
    return distances

In [222]:
def get_tpr_fpr(true_labels, pred_labels):
    TP = np.sum(np.logical_and(pred_labels == 1, true_labels == 1))
    FP = np.sum(np.logical_and(pred_labels == 1, true_labels == 0))

    AP = np.sum(true_labels)
    AN = np.sum(1-true_labels)

    tpr = TP/AP if AP>0 else np.nan
    fpr = FP/AN if AN>0 else np.nan

    return tpr, fpr, TP, AP, FP, AN

In [223]:
def evaluate_test(model, x_test, threshold, dataset):
    Y_all = np.concatenate([np.ones(len(x_test), dtype=bool)])
    #print(Y_all)
    Y_all_pred, Y_all_pred_score = test(model, dataset, x_test, threshold)
    tpr, fpr, tp, ap, fp, an = get_tpr_fpr(Y_all, Y_all_pred)
    return tpr
                                           

# MNIST thresholds

In [224]:
import pandas as pd
import numpy as np

In [225]:
#train data
df_mnist = pd.read_csv("mnist/Benign_MNIST_Metrics.csv")

In [226]:
df_mnist

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059
0,reconstruction error l1,0.015105,0.015935,0.012748,0.006415,0.012948,0.016538,0.006377,0.015353,0.004722,...,0.016145,0.015428,0.016076,0.01402,0.013324,0.013312,0.009356,0.016205,0.01699,0.00804


In [227]:
recon_error_benign = df_mnist.iloc[0].values.flatten().tolist()[1:]

In [228]:
min(recon_error_benign), max(recon_error_benign)

(0.0036156065762043, 0.0322329066693782)

In [229]:
fpr = [0.01, 0.05, 0.1] 
for train_fpr in fpr: 
    print(train_fs(recon_error_benign, train_fpr))

0.024143386632204
0.0202457662671804
0.0184789914637804


In [230]:
threshold = [0.0227928031235933, 0.0195839405059814, 0.0180920865386724]

# now compute TPR

In [231]:
def compute_tpr(adv, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in adv: 
        if value>threshold:
            TP += 1
        else: 
            FN += 1
    
    
    return (TP/(TP+FN))*100

In [232]:
#pgd with five epsilons
df_pgd_eps1 = pd.read_csv('mnist/pgd/eps8_255.csv')
df_pgd_eps2 = pd.read_csv('mnist/pgd/eps16_255.csv')
df_pgd_eps3 = pd.read_csv('mnist/pgd/eps32_255.csv')
df_pgd_eps4 = pd.read_csv('mnist/pgd/eps64_255.csv')


#fgsm with five epsilons
df_fgsm_eps1 = pd.read_csv('mnist/fgsm/eps8_255.csv')
df_fgsm_eps2 = pd.read_csv('mnist/fgsm/eps16_255.csv')
df_fgsm_eps3 = pd.read_csv('mnist/fgsm/eps32_255.csv')
df_fgsm_eps4 = pd.read_csv('mnist/fgsm/eps64_255.csv')

#bim with five epsilons
df_bim_eps1 = pd.read_csv('mnist/bim/eps8_255.csv')
df_bim_eps2 = pd.read_csv('mnist/bim/eps16_255.csv')
df_bim_eps3 = pd.read_csv('mnist/bim/eps32_255.csv')
df_bim_eps4 = pd.read_csv('mnist/bim/eps64_255.csv')

#cw with four confidences
df_cw_conf1 = pd.read_csv('mnist/cw/conf_0.csv')
df_cw_conf2 = pd.read_csv('mnist/cw/conf_50.csv')

In [233]:
magnet1 = df_pgd_eps1.iloc[0].values.flatten().tolist()[1:]
magnet2 = df_pgd_eps2.iloc[0].values.flatten().tolist()[1:]
magnet3 = df_pgd_eps3.iloc[0].values.flatten().tolist()[1:]
magnet4 = df_pgd_eps4.iloc[0].values.flatten().tolist()[1:]

In [234]:
for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    TPR = compute_tpr(magnet3, th)
    print(TPR)
    TPR = compute_tpr(magnet4, th)
    print(TPR)
    print('---')

87.80841799709724
100.0
100.0
100.0
---
99.85486211901306
100.0
100.0
100.0
---
100.0
100.0
100.0
100.0
---


In [235]:
magnet1 = df_fgsm_eps1.iloc[0].values.flatten().tolist()[1:]
magnet2 = df_fgsm_eps2.iloc[0].values.flatten().tolist()[1:]
magnet3 = df_fgsm_eps3.iloc[0].values.flatten().tolist()[1:]
magnet4 = df_fgsm_eps4.iloc[0].values.flatten().tolist()[1:]


for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    TPR = compute_tpr(magnet3, th)
    print(TPR)
    TPR = compute_tpr(magnet4, th)
    print(TPR)
    print('---')

82.04365079365078
100.0
100.0
100.0
---
95.43650793650794
100.0
100.0
100.0
---
98.31349206349206
100.0
100.0
100.0
---


In [236]:
magnet1 = df_bim_eps1.iloc[0].values.flatten().tolist()[1:]
magnet2 = df_bim_eps2.iloc[0].values.flatten().tolist()[1:]
magnet3 = df_bim_eps3.iloc[0].values.flatten().tolist()[1:]
magnet4 = df_fgsm_eps4.iloc[0].values.flatten().tolist()[1:]


for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    TPR = compute_tpr(magnet3, th)
    print(TPR)
    TPR = compute_tpr(magnet4, th)
    print(TPR)
    print('---')

50.69721115537848
100.0
100.0
100.0
---
82.96812749003985
100.0
100.0
100.0
---
90.5378486055777
100.0
100.0
100.0
---


In [239]:
magnet1 = df_cw_conf1.iloc[0].values.flatten().tolist()[1:]
magnet2 = df_cw_conf2.iloc[0].values.flatten().tolist()[1:]

for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    print('----')

3.592814371257485
3.592814371257485
----
9.980039920159681
9.980039920159681
----
17.764471057884233
17.764471057884233
----


# Check test FPR

In [240]:
df_mnist = pd.read_csv("mnist/Test_Benign_MNIST_Metrics.csv")

In [241]:
recon = df_mnist.iloc[0].values.flatten().tolist()[1:]
threshold = [0.0227928031235933, 0.0195839405059814, 0.0180920865386724]

In [242]:
def compute_FPR(ben, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in ben: 
        if value<threshold:
            TN += 1
        else: 
            FP += 1
    
    
    return (FP/(TN+FP))*100

In [243]:
for th in threshold:
    FPR = compute_FPR(recon, th)
    print(FPR)

1.5686274509803921
6.176470588235294
10.196078431372548


# Compute ROC AUC score

In [244]:
df_mnist = pd.read_csv("mnist/Test_Benign_MNIST_Metrics.csv")
recon = df_mnist.iloc[0].values.flatten().tolist()[1:]

In [245]:
min(recon), max(recon)

(0.0029677408747375, 0.0278869736939668)

In [246]:
threshold = [0.0227928031235933, 0.0195839405059814, 0.0180920865386724, 0.017, 0.016, 0.015, 0.013, 0.012, 0.011, 0.004, 0.003]

In [107]:
fpr_results = []
for th in threshold:
    FPR = compute_FPR(recon, th)
    fpr_results.append(FPR/100)
print(fpr_results)

[0.01568627450980392, 0.061764705882352944, 0.10196078431372549, 0.1568627450980392, 0.22058823529411764, 0.2950980392156863, 0.4843137254901961, 0.6049019607843137, 0.7049019607843138, 0.9970588235294118, 0.9990196078431373]


In [247]:
#pgd with five epsilons
df_pgd_eps1 = pd.read_csv('mnist/pgd/eps8_255.csv')
df_pgd_eps2 = pd.read_csv('mnist/pgd/eps16_255.csv')
df_pgd_eps3 = pd.read_csv('mnist/pgd/eps32_255.csv')
df_pgd_eps4 = pd.read_csv('mnist/pgd/eps64_255.csv')


#fgsm with five epsilons
df_fgsm_eps1 = pd.read_csv('mnist/fgsm/eps8_255.csv')
df_fgsm_eps2 = pd.read_csv('mnist/fgsm/eps16_255.csv')
df_fgsm_eps3 = pd.read_csv('mnist/fgsm/eps32_255.csv')
df_fgsm_eps4 = pd.read_csv('mnist/fgsm/eps64_255.csv')

#bim with five epsilons
df_bim_eps1 = pd.read_csv('mnist/bim/eps8_255.csv')
df_bim_eps2 = pd.read_csv('mnist/bim/eps16_255.csv')
df_bim_eps3 = pd.read_csv('mnist/bim/eps32_255.csv')
df_bim_eps4 = pd.read_csv('mnist/bim/eps64_255.csv')

#cw with four confidences
df_cw_conf1 = pd.read_csv('mnist/cw/conf_0.csv')
df_cw_conf2 = pd.read_csv('mnist/cw/conf_50.csv')

In [248]:
def compute_TPR(adv, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in adv: 
        if value>threshold:
            TP += 1
        else: 
            FN += 1
    
    
    return (TP/(TP+FN))*100

In [249]:
def compute_auc(adv_list, th): 
    tpr_results = []
    for th in threshold:
        TPR = compute_TPR(adv_list, th)
        tpr_results.append(TPR/100)
    return sklearn.metrics.auc(fpr_results, tpr_results)

In [250]:
import sklearn
from sklearn.metrics import roc_auc_score

feature_squeezer_1 = df_pgd_eps1.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_2 = df_pgd_eps2.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_3 = df_pgd_eps3.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_4 = df_pgd_eps4.iloc[0].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))
print(compute_auc(feature_squeezer_3, threshold))
print(compute_auc(feature_squeezer_4, threshold))

0.980461879962435
0.9833333333333333
0.9833333333333333
0.9833333333333333


In [251]:
feature_squeezer_1 = df_fgsm_eps1.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_2 = df_fgsm_eps2.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_3 = df_fgsm_eps3.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_4 = df_fgsm_eps4.iloc[0].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))
print(compute_auc(feature_squeezer_3, threshold))
print(compute_auc(feature_squeezer_4, threshold))

0.9754911686896981
0.9833333333333333
0.9833333333333333
0.9833333333333333


In [252]:
feature_squeezer_1 = df_bim_eps1.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_2 = df_bim_eps2.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_3 = df_bim_eps3.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_4 = df_bim_eps4.iloc[0].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))
print(compute_auc(feature_squeezer_3, threshold))
print(compute_auc(feature_squeezer_4, threshold))

0.9553570033591126
0.9833333333333333
0.9833333333333333
0.9833333333333333


In [255]:
feature_squeezer_1 = df_cw_conf1.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_2 = df_cw_conf2.iloc[0].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))

0.5821660600367893
0.5821660600367893
